In [1]:
import pandas as pd
import numpy as np
import re
#import math
#from random import choices
#import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
#from matplotlib.colors import ListedColormap
#import statistics

In [2]:
import devanalyst.simulation.statics as S_

from devanalyst.simulation.businessObjects import WorkAssignments, ReleaseCycleContext
from devanalyst.simulation.simulationModels import BalancedAllocationModel, GreedyAllocationModel, ModelsConfig, \
Distribution, DefaultCostModel, QualityModel

import devanalyst.simulation.generateTimecards as timecard
#from devanalyst.simulation.generateTimecards import WorkAssignments


importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\statics.ipynb
importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\businessObjects.ipynb
importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\simulationModels.ipynb
importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\generateTimecards.ipynb


In [3]:
import devanalyst.simulation.tests.test_utils as tu_

importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\tests\test_utils.ipynb


In [4]:
import devanalyst.simulation.visualizations.simm_visuals as simm_visuals
import devanalyst.simulation.visualizations.timecard_visuals as tc_visuals

importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\visualizations\simm_visuals.ipynb
importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\visualizations\timecard_visuals.ipynb


<h1>Pilots</h1>

<h2>Bug inflow prototypes</h2>

In [5]:
class DistributedLagQualityModel(QualityModel):

    def __init__(self):
        super(DistributedLagQualityModel, self).__init__()
        return
    
    # Returns a list of Ticket instances.
    # -modelsConfig: a ModelsConfig instance
    def findBugs(self, modelsConfig): 
        uss_list = DistributedLagQualityModel._findFinishedStories(modelsConfig.context)
        
        bugs = []
        current_sprint = modelsConfig.context.sprint
        for uss in uss_list:
            delivery_sprint = uss.sprintDelivered
            lag = current_sprint - delivery_sprint
            story = modelsConfig.context.storiesRepo.findStory(uss.userStoryId)
            story_bugs = DistributedLagQualityModel._findDefectsInStory(lag, story, modelsConfig)
            bugs.extend(story_bugs)
        return bugs

    # Returns a list of UserStoryStatus instances, corresponding to all user stories that have been finished.
    def _findFinishedStories(context):
        uss_list = []
        ids = context.storiesRepo.findIds()
        for userStoryId in ids:
            uss = context.teamsRepo.getUserStoryStatus(userStoryId)
            if (uss.percentAchieved == 1.0):
                uss_list.append(uss)
        return uss_list
    
    # Returns a list of Ticket instances
    def _findDefectsInStory(lag, userStory, modelsConfig):
        if lag < 1 or lag > 3: # No bugs for stories just finished or which were finished a while ago
            return []
        # Exposure is the probability of finding a bug. For now hardcode a 50% exposure distributed over 
        # 3 sprints with the surge in the middle one
        if lag == 1:
            exposure = 0.125
        if lag == 2:
            exposure = 0.25
        if lag == 3:
            exposure = 0.125
        possible_defect_count  = [0, 1] # Possible values for number of bugs found
        likelihoods            = [1-exposure, exposure]
        defect_count           = modelsConfig.random.pickOneWithWeights(possible_defect_count, likelihoods)
        
        # Issue defect Tickets
        defects = []
        repo = modelsConfig.context.ticketsRepo
        for i in range(defect_count):
            costToFix = DistributedLagQualityModel._estimateCostToFix(userStory, modelsConfig)
            ticket = repo.addTicket(userStory.userStoryId, costToFix)
            defects.append(ticket)
        return defects
    
        
    def _estimateCostToFix(userStory, modelsConfig):
    # For now hard-code a simplistic cost to fix: each bug costs 20% of the original estimate of the story.
    # Better would be a percentage of the actual cost to develop.
         return 0.20 * userStory.originalEstimate


In [6]:
def testTicketInflow(modelsConfig):
    RELEASE_DURATION = 60
    SPRINT_DURATION = 10
    
    tu_.loadTestResources()
    
    teams_df, stories_df, teamsRepo, storiesRepo, ticketsRepo = tu_.initTestData(tu_.DEV_DF, \
                                                                                           tu_.PM_DF, \
                                                                                           RELEASE_DURATION, \
                                                                                           SPRINT_DURATION, \
                                                                                          modelsConfig)
    TEAM_ID = 'Team A'
    SPRINT = 1    
    ctx = ReleaseCycleContext(TEAM_ID, teamsRepo, storiesRepo, ticketsRepo, SPRINT, SPRINT_DURATION)
    modelsConfig.context = ctx

    # Pretend all stories have been completed on the first sprint
    uss_list = []
    ids = ctx.storiesRepo.findIds()
    for userStoryId in ids:
        uss = ctx.teamsRepo.getUserStoryStatus(userStoryId)
        uss.percentAchieved = 1.0
        uss.planned = True
        uss.sprintPlanned = 1
        uss.sprintDelivered = 1
    
    # Now pretend we raverse the next 5 sprints, generating bugs
    bugs = []
    for i in range(5):
        ctx.sprint = SPRINT + i + 1
        qualityModel = modelsConfig.qualityModels[0]
        bugs.extend(qualityModel.findBugs(modelsConfig))
    return bugs

In [7]:
# -bugs: a list of Tickets
def build_bugs_df(bugs):
    bugs_dict = {}
    bugs_dict['Ticket Id']        = []
    bugs_dict['User Story Id']    = []
    bugs_dict['Cost to Fix']      = []
    bugs_dict['Effort to Date']   = []
    bugs_dict['Percent Achieved'] = []
    
    for bug in bugs:
        bugs_dict['Ticket Id']        .append(bug.ticketId)
        bugs_dict['User Story Id']    .append(bug.userStoryId)
        bugs_dict['Cost to Fix']      .append(bug.costToFix)
        bugs_dict['Effort to Date']   .append(bug.effortToDate)
        bugs_dict['Percent Achieved']  .append(bug.percentAchieved)
        
    bugs_df = pd.DataFrame(bugs_dict)
    return bugs_df

In [8]:
# -bmodelsConfig
def build_stories_df(modelsConfig):
    stories_dict = {}
    stories_dict['User Story Id']          = []
    stories_dict['Original Estimate']  = []
    stories_dict['Developer']          = []
    stories_dict['Product Manager']    = []
    
    repo = modelsConfig.context.storiesRepo
    for storyId in repo.findIds():
        story = repo.findStory(storyId)
        stories_dict['User Story Id']       .append(story.userStoryId)
        stories_dict['Original Estimate']   .append(story.originalEstimate)
        stories_dict['Developer']           .append(story.developer)
        stories_dict['Product Manager']     .append(story.productManager)
        
    stories_df = pd.DataFrame(stories_dict)
    return stories_df

In [9]:
modelsConfig = ModelsConfig([DefaultCostModel(0.25)], [DistributedLagQualityModel()], GreedyAllocationModel()) 
modelsConfig.random.reset(271)
bugs = testTicketInflow(modelsConfig)
bugs_df = build_bugs_df(bugs)
bugs_df[:6]

,Ticket Id,User Story Id,Cost to Fix,Effort to Date,Percent Achieved
0,Ticket #1,UserStory #3,0.8,0.0,0.0
1,Ticket #2,UserStory #10,1.8,0.0,0.0
2,Ticket #3,UserStory #15,0.6,0.0,0.0
3,Ticket #4,UserStory #22,0.6,0.0,0.0
4,Ticket #5,UserStory #30,0.4,0.0,0.0
5,Ticket #6,UserStory #36,2.0,0.0,0.0


In [10]:
bugs_df['Cost to Fix'].sum()

160.2

In [11]:
stories_df = build_stories_df(modelsConfig)
stories_df[:6]

,User Story Id,Original Estimate,Developer,Product Manager
0,UserStory #1,10,Beau Hockensmith,Sherlyn Cordle
1,UserStory #2,10,Glenna Mcghie,Sherlyn Cordle
2,UserStory #3,4,Francisco Hoppe,Edgar Hibbler
3,UserStory #4,7,Beau Hockensmith,Sherlyn Cordle
4,UserStory #5,3,Gregorio Darr,Edgar Hibbler
5,UserStory #6,3,Beau Hockensmith,Sherlyn Cordle


In [12]:
stories_df['Original Estimate'].sum()

1686

In [13]:
def countUniques(seriesGroup):
    if (seriesGroup.unique()[0] == None):
        return 0
    else:
        return seriesGroup.unique().size

In [14]:
countUniques(bugs_df['User Story Id'])

132

In [59]:
bugs_df.shape

(155, 5)

In [15]:
modelsConfig.context.sprint

6

In [23]:
CTX = modelsConfig.context
TEAM_ID = 'Team A'
work = WorkAssignments(TEAM_ID, CTX.teamsRepo, CTX.storiesRepo, 7)

In [24]:
work.committedTasks()

,Owner,Task Type,User Story Id,Planned for Sprint,Delivered in Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved


In [25]:
team = CTX.teamsRepo.findTeam(TEAM_ID)

In [28]:
team.backlog.pendingUserStories[0]

In [32]:
CTX.teamsRepo.getUserStoryStatus('UserStory #3').pendingTickets

[]

In [ ]:
def genReleaseCycleSheets(modelsConfig):
    RELEASE_DURATION = 60
    SPRINT_DURATION = 10
    
    tu_.loadTestResources()
    
    teams_df, stories_df, teamsRepo, storiesRepo, ticketsRepo = tu_.initTestData(tu_.DEV_DF, \
                                                                                           tu_.PM_DF, \
                                                                                           RELEASE_DURATION, \
                                                                                           SPRINT_DURATION, \
                                                                                          modelsConfig)
    
    NUMBER_OF_SPRINTS = 25

    entries_df, releaseLog = timecard.runReleaseCycle(teamsRepo, ticketsRepo, storiesRepo, datetime(2018, 1, 15), \
                                                   SPRINT_DURATION, NUMBER_OF_SPRINTS, modelsConfig)    
    
    return entries_df, releaseLog, teamsRepo, storiesRepo

In [ ]:
SPRINT_DURATION = 10
modelsConfig = ModelsConfig([DefaultCostModel(0.25)], [DistributedLagQualityModel()], GreedyAllocationModel()) 
modelsConfig.random.reset(271)
entries_df, releaseLog, TEAMS_REPO, STORIES_REPO = genReleaseCycleSheets(modelsConfig)

In [ ]:
tc_visuals.renderReleaseCycleLog('Team A', releaseLog, 1, 4)

In [ ]:
end_df = releaseLog.log['Team A'][1]['planned_End_CURRENT_SPRINT']
end_df

<h2>Burnout in release cycle - Pilot</h2>

In [ ]:
# 
def genReleaseCycle(modelsConfig):
    RELEASE_DURATION = 60
    SPRINT_DURATION = 10
    
    tu_.loadTestResources()
    
    teams_df, stories_df, teamsRepo, storiesRepo, ticketsRepo = tu_.initTestData(tu_.DEV_DF, \
                                                                                           tu_.PM_DF, \
                                                                                           RELEASE_DURATION, \
                                                                                           SPRINT_DURATION, \
                                                                                          modelsConfig)
    
    NUMBER_OF_SPRINTS = 25

    entries_df, worksheets = timecard.runReleaseCycle(teamsRepo, ticketsRepo, storiesRepo, datetime(2018, 1, 15), \
                                                   SPRINT_DURATION, NUMBER_OF_SPRINTS, modelsConfig)    
    
    return entries_df, worksheets, storiesRepo

In [ ]:
def genBurnout(entries_df, storiesRepo):
    bystory = entries_df.groupby('User Story')
    u = bystory.apply(storyInfo, storiesRepo)
    
    
    u = u.reset_index()
    u.drop(['level_1'], axis='columns', inplace=True)
    s = u.groupby('Final Sprint')
    burnout = s.apply(sprintInfo)
    burnout = burnout.reset_index()
    burnout.drop(['level_1'], axis='columns', inplace=True)
    

    return burnout, u

In [ ]:
def storyInfo(group_df, storiesRepo):
    info = {}
    
    #Go back, as min['Date'] is end of first sprint
    info['Start'] = [timecard.subtractBusinessDays(group_df['Date'].min(), SPRINT_DURATION)] 
    info['End'] = [group_df['Date'].max()]
    info['Elapsed Time'] = (info['End'][0] - info['Start'][0]).days
    info['Effort'] = group_df['Time Spent'].sum()
    info['Initial Sprint'] = [group_df['Sprint'].min()]
    info['Final Sprint'] = [group_df['Sprint'].max()]  
    '''
    userStoryId = group_df['User Story']
    userStory = storiesRepo.findStory(userStoryId)
    info['Original estimate'] = userStory.originalEstimate    
    '''
    
    return pd.DataFrame(info)

In [ ]:
def sprintInfo(group_df):
    info = {}
    
    info['Effort'] = group_df['Effort'].sum()
    info['Avg Effort'] = group_df['Effort'].mean()
    info['Stories Completed'] = [group_df['User Story'].count()]
    info['Over 1 sprint'] = [group_df[group_df['Elapsed Time']==14]['User Story'].count()]
    info['Over 2 sprint'] = [group_df[group_df['Elapsed Time']==28]['User Story'].count()]
    info['Over 3 sprint'] = [group_df[group_df['Elapsed Time']==42]['User Story'].count()]
    info['Over 4 sprint'] = [group_df[group_df['Elapsed Time']==56]['User Story'].count()]
 
    
    return pd.DataFrame(info)

In [ ]:
SPRINT_DURATION = 10
modelsConfig1 = ModelsConfig([DefaultCostModel(0.0)], [], BalancedAllocationModel(SPRINT_DURATION)) 
modelsConfig1.random.reset(271)
entries_df1, worksheets1, storiesRepo = genReleaseCycle(modelsConfig1)

In [ ]:
burn1, u1 = genBurnout(entries_df1, storiesRepo)
burn1

In [ ]:
u1

In [ ]:
SPRINT_DURATION = 10
modelsConfig2 = ModelsConfig([DefaultCostModel(0.0)], [], GreedyAllocationModel(SPRINT_DURATION)) 
modelsConfig2.random.reset(271)
entries_df2, worksheets2, storiesRepo = genReleaseCycle(modelsConfig2)

In [ ]:
burn2, u2 = genBurnout(entries_df2, storiesRepo)
burn2

In [ ]:
burn1['Effort'].plot(label = 'Balanced'), burn2['Effort'].plot(label = 'Greedy'), plt.legend()

In [ ]:
burn1['Avg Effort'].plot(label = 'Balanced'), burn2['Avg Effort'].plot(label = 'Greedy'), plt.legend()

In [ ]:
u1_10 = u1[u1['Final Sprint'] == 10]
u1_10.shape

In [ ]:
u1_10[u1_10['Initial Sprint'] == 10]['Effort'].mean()

In [ ]:
u1_10['Effort'].mean()

In [ ]:
u1_10[u1_10['Initial Sprint'] == 10].shape

In [ ]:
u1_10[u1_10['Initial Sprint'] == 9]['Effort'].mean()

In [ ]:
u1_10[u1_10['Initial Sprint'] == 9].shape